In [1]:
# Colabis kasutamiseks
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/thesis/')

!pip install estnltk==1.7.4
!pip install evaluate
!pip install seqeval
!pip install nervaluate

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-no

In [46]:
from data_processing import DatasetProcessor
from bert_data_processing import BERTDataProcessor
from bert_evaluator import BERTEvaluator
from bert_trainer import BERTTrainer
ALL_TAGS = DatasetProcessor.ALL_TAGS
TAG2IDX = DatasetProcessor.TAG2IDX
IDX2TAG = DatasetProcessor.IDX2TAG
from transformers import BertTokenizer, BertForTokenClassification, pipeline, Trainer, TrainingArguments

In [54]:
def evaluate_pretrained(model_name):
  model = BertForTokenClassification.from_pretrained(model_name)
  bert_processor = BERTDataProcessor(model_name)

  label2id = model.config.label2id
  ewt_processor = DatasetProcessor('ewt', from_json=True)
  ewt_test = ewt_processor.test

  edt_processor = DatasetProcessor('edt', from_json=True)
  edt_test = edt_processor.test

  if model_name.lower() == 'estbert_ner_v2':
    IDX2TAG[3] = 'B-GPE'
    IDX2TAG[4] = 'I-GPE'

  def convert_to_model_id(example):
    example['tags'] = [IDX2TAG[tag] for tag in example['tags']]
    example['tags'] = [label2id.get(tag, label2id['O']) for tag in example['tags']]
    return example

  ewt_test = ewt_test.map(convert_to_model_id)
  edt_test = edt_test.map(convert_to_model_id)

  ewt_test = bert_processor.tokenize_dataset(ewt_test)
  edt_test = bert_processor.tokenize_dataset(edt_test)
  label2id = list(label2id.keys())
  #label2id.sort()
  evaluator = BERTEvaluator(all_tags=label2id, ner_tags=list(set([label[2:] for label in label2id if label != 'O'])))

  training_args = TrainingArguments(
            report_to='none',
            output_dir='./results',
            #per_device_eval_batch_size=16
        )
  trainer = Trainer(
            model=model,
            processing_class=bert_processor.tokenizer,
            data_collator=bert_processor.data_collator,
            compute_metrics=evaluator.compute_metrics,
            args=training_args
        )
  print(f'{model_name} EWT testandmestikul')
  ewt_test_results = evaluator.evaluate_and_print(ewt_test, trainer)

  print(f'{model_name} EDT testandmestikul')
  edt_test_results = evaluator.evaluate_and_print(edt_test, trainer)
  print()

In [55]:
evaluate_pretrained('tartuNLP/EstBERT_NER')

Some weights of the model checkpoint at tartuNLP/EstBERT_NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Map:   0%|          | 0/913 [00:00<?, ? examples/s]

Map:   0%|          | 0/3207 [00:00<?, ? examples/s]

Map:   0%|          | 0/913 [00:00<?, ? examples/s]

Map:   0%|          | 0/3207 [00:00<?, ? examples/s]

tartuNLP/EstBERT_NER EWT testandmestikul
Hindan testandmestikul..


Seqeval tulemused
LOC {'precision': 0.20689655172413793, 'recall': 0.32727272727272727, 'f1': 0.2535211267605634, 'number': 55}
ORG {'precision': 0.27884615384615385, 'recall': 0.12554112554112554, 'f1': 0.17313432835820897, 'number': 231}
PER {'precision': 0.7008086253369272, 'recall': 0.2374429223744292, 'f1': 0.35470668485675305, 'number': 1095}
overall_precision 0.5462633451957295
overall_recall 0.222302679217958
overall_f1 0.31600617601646935
overall_accuracy 0.9305341331048272

Nervaluate tulemused
Strict {'correct': 307, 'incorrect': 253, 'partial': 0, 'missed': 821, 'spurious': 118, 'possible': 1381, 'actual': 678, 'precision': 0.4528023598820059, 'recall': 0.222302679217958, 'f1': 0.2982030111704711}
precision 0.4528023598820059
recall 0.222302679217958
f1 0.2982030111704711
PER {'correct': 260, 'incorrect': 131, 'partial': 0, 'missed': 704, 'spurious': 32, 'possible': 1095, 'actual': 423, 'precision': 0.6146572104018913, 'recall': 0.2374429223744292, 'f1': 0.3425559947299078}

Seqeval tulemused
LOC {'precision': 0.3459715639810427, 'recall': 0.4965986394557823, 'f1': 0.40782122905027934, 'number': 441}
ORG {'precision': 0.3333333333333333, 'recall': 0.23593964334705075, 'f1': 0.2763052208835341, 'number': 729}
PER {'precision': 0.6501672240802676, 'recall': 0.41770519982810483, 'f1': 0.5086342229199372, 'number': 2327}
overall_precision 0.5155068078668684
overall_recall 0.38976265370317414
overall_f1 0.44390164468327636
overall_accuracy 0.954740330103491

Nervaluate tulemused
Strict {'correct': 1363, 'incorrect': 1136, 'partial': 0, 'missed': 998, 'spurious': 698, 'possible': 3497, 'actual': 3197, 'precision': 0.4263371911166719, 'recall': 0.38976265370317414, 'f1': 0.4072303555422767}
precision 0.4263371911166719
recall 0.38976265370317414
f1 0.4072303555422767
PER {'correct': 972, 'incorrect': 614, 'partial': 0, 'missed': 741, 'spurious': 206, 'possible': 2327, 'actual': 1792, 'precision': 0.5424107142857143, 'recall': 0.41770519982810483, 'f1': 0.47195921

In [56]:
evaluate_pretrained('tartuNLP/EstBERT_NER_V2')

Map:   0%|          | 0/913 [00:00<?, ? examples/s]

Map:   0%|          | 0/3207 [00:00<?, ? examples/s]

Map:   0%|          | 0/913 [00:00<?, ? examples/s]

Map:   0%|          | 0/3207 [00:00<?, ? examples/s]

tartuNLP/EstBERT_NER_V2 EWT testandmestikul
Hindan testandmestikul..


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seqeval tulemused
DATE {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
EVENT {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
GPE {'precision': 0.22105263157894736, 'recall': 0.38181818181818183, 'f1': 0.27999999999999997, 'number': 55}
LOC {'precision': 0.14285714285714285, 'recall': 0.09090909090909091, 'f1': 0.1111111111111111, 'number': 55}
MONEY {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
ORG {'precision': 0.13690476190476192, 'recall': 0.09956709956709957, 'f1': 0.11528822055137845, 'number': 231}
PER {'precision': 0.5152091254752852, 'recall': 0.24748858447488584, 'f1': 0.3343615052436767, 'number': 1095}
PERCENT {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
PROD {'precision': 0.22123893805309736, 'recall': 0.06906077348066299, 'f1': 0.10526315789473685, 'number': 362}
TIME {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
TITLE {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
overall_precision 0.3069395017793594
over

Seqeval tulemused
DATE {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
EVENT {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
GPE {'precision': 0.25, 'recall': 0.19010416666666666, 'f1': 0.21597633136094677, 'number': 384}
LOC {'precision': 0.36619718309859156, 'recall': 0.11791383219954649, 'f1': 0.1783876500857633, 'number': 441}
MONEY {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
ORG {'precision': 0.23769100169779286, 'recall': 0.19204389574759945, 'f1': 0.21244309559939303, 'number': 729}
PER {'precision': 0.48569612205975843, 'recall': 0.3283197249677697, 'f1': 0.3917948717948718, 'number': 2327}
PERCENT {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
PROD {'precision': 0.1884498480243161, 'recall': 0.10508474576271186, 'f1': 0.13492927094668117, 'number': 590}
TIME {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
TITLE {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
overall_precision 0.2868787799105969
overall_recall 0.2